In [1]:
from pyspark.sql import SparkSession
import pandas as pd

In [2]:
spark = SparkSession.builder.appName('practice').getOrCreate()
spark

24/07/16 02:34:29 WARN Utils: Your hostname, mennas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.3 instead (on interface en0)
24/07/16 02:34:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/16 02:34:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
train_df = spark.read.csv('train.csv', header=True, inferSchema=True)
train_df

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]

In [4]:
train_df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
|          6|       0|     3|    Moran, Mr. James|  male|NULL|    0|    0|      

#### Filter Operation

In [6]:
# to filter on a condition
train_df.filter('Age>25').show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|54.0|    0|    0|           17463|51.8625|  E46|       S|
|          9|       1|     3|Johnson, Mrs. Osc...|female|27.0|    0|    2|      

In [11]:
# after filter on specific condition you can choose specific columns to be shown
train_df.filter('Survived==1').select(['Name', 'Ticket']).show()

+--------------------+----------------+
|                Name|          Ticket|
+--------------------+----------------+
|Cumings, Mrs. Joh...|        PC 17599|
|Heikkinen, Miss. ...|STON/O2. 3101282|
|Futrelle, Mrs. Ja...|          113803|
|Johnson, Mrs. Osc...|          347742|
|Nasser, Mrs. Nich...|          237736|
|Sandstrom, Miss. ...|         PP 9549|
|Bonnell, Miss. El...|          113783|
|Hewlett, Mrs. (Ma...|          248706|
|Williams, Mr. Cha...|          244373|
|Masselmani, Mrs. ...|            2649|
|Beesley, Mr. Lawr...|          248698|
|"McGowan, Miss. A...|          330923|
|Sloper, Mr. Willi...|          113788|
|Asplund, Mrs. Car...|          347077|
|"O'Dwyer, Miss. E...|          330959|
|Spencer, Mrs. Wil...|        PC 17569|
|Glynn, Miss. Mary...|          335677|
|    Mamee, Mr. Hanna|            2677|
|Nicola-Yarred, Mi...|            2651|
|Laroche, Miss. Si...|   SC/Paris 2123|
+--------------------+----------------+
only showing top 20 rows



In [20]:
# to filter on multiple conditions
train_df.filter((train_df['Survived']==1) & (train_df['Sex']=='female')).show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+--------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|    Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+--------+-----+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599| 71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|   7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|    53.1| C123|       S|
|          9|       1|     3|Johnson, Mrs. Osc...|female|27.0|    0|    2|          347742| 11.1333| NULL|       S|
|         10|       1|     2|Nasser, Mrs. Nich...|female|14.0|    1|    0|          237736| 30.0708| NULL|       C|
|         11|       1|     3|Sandstrom, Miss. ...|female| 4.0|    1|    

#### GroupBy Operation

In [21]:
car_prices_df = spark.read.csv('CarPrices.csv', header=True, inferSchema=True)
car_prices_df

DataFrame[car_ID: int, symboling: int, CarName: string, fueltype: string, aspiration: string, doornumber: string, carbody: string, drivewheel: string, enginelocation: string, wheelbase: double, carlength: double, carwidth: double, carheight: double, curbweight: int, enginetype: string, cylindernumber: string, enginesize: int, fuelsystem: string, boreratio: double, stroke: double, compressionratio: double, horsepower: int, peakrpm: int, citympg: int, highwaympg: int, price: double]

In [22]:
car_prices_df.printSchema()

root
 |-- car_ID: integer (nullable = true)
 |-- symboling: integer (nullable = true)
 |-- CarName: string (nullable = true)
 |-- fueltype: string (nullable = true)
 |-- aspiration: string (nullable = true)
 |-- doornumber: string (nullable = true)
 |-- carbody: string (nullable = true)
 |-- drivewheel: string (nullable = true)
 |-- enginelocation: string (nullable = true)
 |-- wheelbase: double (nullable = true)
 |-- carlength: double (nullable = true)
 |-- carwidth: double (nullable = true)
 |-- carheight: double (nullable = true)
 |-- curbweight: integer (nullable = true)
 |-- enginetype: string (nullable = true)
 |-- cylindernumber: string (nullable = true)
 |-- enginesize: integer (nullable = true)
 |-- fuelsystem: string (nullable = true)
 |-- boreratio: double (nullable = true)
 |-- stroke: double (nullable = true)
 |-- compressionratio: double (nullable = true)
 |-- horsepower: integer (nullable = true)
 |-- peakrpm: integer (nullable = true)
 |-- citympg: integer (nullable = t

In [23]:
# I am going to select some columns to work with
car_prices_df = car_prices_df.select(['car_ID', 'CarName','fueltype', 'price' ])
car_prices_df.show()

+------+--------------------+--------+---------+
|car_ID|             CarName|fueltype|    price|
+------+--------------------+--------+---------+
|     1|  alfa-romero giulia|     gas|  13495.0|
|     2| alfa-romero stelvio|     gas|  16500.0|
|     3|alfa-romero Quadr...|     gas|  16500.0|
|     4|         audi 100 ls|     gas|  13950.0|
|     5|          audi 100ls|     gas|  17450.0|
|     6|            audi fox|     gas|  15250.0|
|     7|          audi 100ls|     gas|  17710.0|
|     8|           audi 5000|     gas|  18920.0|
|     9|           audi 4000|     gas|  23875.0|
|    10| audi 5000s (diesel)|     gas|17859.167|
|    11|            bmw 320i|     gas|  16430.0|
|    12|            bmw 320i|     gas|  16925.0|
|    13|              bmw x1|     gas|  20970.0|
|    14|              bmw x3|     gas|  21105.0|
|    15|              bmw z4|     gas|  24565.0|
|    16|              bmw x4|     gas|  30760.0|
|    17|              bmw x5|     gas|  41315.0|
|    18|            

In [24]:
# to show the sum of prices of each fuel type
car_prices_df.groupBy('fueltype').sum().show()

+--------+-----------+-----------+
|fueltype|sum(car_ID)| sum(price)|
+--------+-----------+-----------+
|     gas|      18603|2404962.667|
|  diesel|       2512|   316763.0|
+--------+-----------+-----------+



In [29]:
# to show the average price of each car name
car_prices_df.groupBy('CarName').mean().show()

+--------------------+-----------+------------------+
|             CarName|avg(car_ID)|        avg(price)|
+--------------------+-----------+------------------+
|         audi 100 ls|        4.0|           13950.0|
|       dodge rampage|       22.0|            5572.0|
|       mitsubishi g4|       82.0|10352.333333333334|
|        nissan teana|      105.0|           17199.0|
|         nissan note|       98.0|            7999.0|
|    maxda glc deluxe|       52.0|            6095.0|
|         volvo 244dl|      200.0|           18735.0|
|          isuzu MU-X|       44.0|            6785.0|
|              bmw x3|       16.0|           28992.5|
|       peugeot 604sl|      115.5|           17525.0|
|   honda accord cvcc|       34.0|            6529.0|
| honda civic 1500 gl|       37.0|            7295.0|
|toyota corona har...|      154.0|            6918.0|
|          mazda rx-4|       58.5|            8470.0|
|              bmw z4|       15.0|           24565.0|
|        renault 12tl|      

In [30]:
# to count how many cars in each fuel type
car_prices_df.groupBy('fueltype').count().show()


+--------+-----+
|fueltype|count|
+--------+-----+
|     gas|  185|
|  diesel|   20|
+--------+-----+



In [31]:
# to sum the all prices of all cars
car_prices_df.agg({'price':'sum'}).show()

+-----------+
| sum(price)|
+-----------+
|2721725.667|
+-----------+



In [32]:
car_prices_df.agg({'price':'max'}).show()


+----------+
|max(price)|
+----------+
|   45400.0|
+----------+



In [33]:
car_prices_df.groupBy('fueltype').max().show()

+--------+-----------+----------+
|fueltype|max(car_ID)|max(price)|
+--------+-----------+----------+
|     gas|        205|   45400.0|
|  diesel|        204|   31600.0|
+--------+-----------+----------+

